In [87]:
import pandas as pd

In [88]:
MODEL_COLUMN = "model"
BASELINE="BEVfusion"
CORRUPTIONS = ["beamsreducing", "brightness", "dark", "fog", "missingcamera", "motionblur", "pointsreducing", "snow", "spatialmisalignment", "temporalmisalignment"]
NICE_NAMES = {"beamsreducing": "Beams Red.",
              "brightness": "Brightness",
              "dark": "Darkness",
              "fog":"Fog",
              "missingcamera":
              "Missing Cam.",
              "motionblur": "Motion Blur",
              "pointsreducing": "Points Red.",
              "snow": "Snow",
              "spatialmisalignment": "Spatial Mis.",
              "temporalmisalignment": "Temporal Mis."
              }
MODELS = ["CMT", "DeepInteraction", "TransFusion", "Sparsefusion", "BEVfusion"]
MODELS_TO_LINKS = {
    "CMT": "https://github.com/junjie18/CMT",
    "DeepInteraction": "https://github.com/fudan-zvg/DeepInteraction",
    "TransFusion": "https://github.com/XuyangBai/TransFusion",
    "Sparsefusion": "https://github.com/yichen928/SparseFusion",
    "BEVfusion": "https://github.com/mit-han-lab/bevfusion"
}
MODELS_TO_LINKS_MD = {k: f"[{k}]({v})" for k, v in MODELS_TO_LINKS.items()}
SEVERITY_LEVELS = ["1", "2", "3"]

In [89]:
df = pd.read_csv("../results/results_nds.csv")

In [90]:
df = df.dropna()

In [91]:
df

,model,clean,temporalmisalignment_s1,temporalmisalignment_s2,temporalmisalignment_s3,spatialmisalignment_s1,spatialmisalignment_s2,spatialmisalignment_s3,pointsreducing_s1,pointsreducing_s2,...,dark_s3,brightness_s1,brightness_s2,brightness_s3,beamsreducing_s1,beamsreducing_s2,beamsreducing_s3,snow_s1,snow_s2,snow_s3
0,CMT,0.7290,0.6572,0.5680,0.4988,0.6462,0.5860,0.5367,0.7257,0.7180,...,0.6786,0.7284,0.7212,0.7037,0.6826,0.5873,0.4497,0.6483,0.6190,0.5554
1,DeepInteraction,0.6909,0.6139,0.5237,0.4544,0.5863,0.5014,0.4267,0.6859,0.6772,...,0.6254,0.6845,0.6767,0.6785,0.6162,0.4571,0.2839,0.5526,0.5311,0.4896
2,TransFusion,0.7084,0.6306,0.5437,0.4766,0.5708,0.4761,0.4095,0.7027,0.6908,...,0.6978,0.7080,0.7069,0.7056,0.6222,0.4591,0.2636,0.5447,0.5353,0.5087
3,Sparsefusion,0.7315,0.6523,0.5622,0.4901,0.6061,0.5149,0.4461,0.7272,0.7175,...,0.6978,0.7302,0.7257,0.7205,0.6593,0.5139,0.3380,0.5859,0.5704,0.5337
4,BEVfusion,0.7144,0.6341,0.5471,0.4085,0.5903,0.4943,0.4266,0.7096,0.6978,...,0.6861,0.7126,0.7053,0.6969,0.6396,0.4899,0.3199,0.5782,0.5572,0.5233


## Resistance Ability (RA)

In [92]:
for c in CORRUPTIONS:
    df[f"RA_{c}_s1"] = df[c+"_s1"] / df['clean']
    df[f"RA_{c}_s2"] = df[c+"_s2"] / df['clean']
    df[f"RA_{c}_s3"] = df[c+"_s3"] / df['clean']
    df[f"RA_{c}"] = (df[f"RA_{c}_s1"] + df[f"RA_{c}_s2"] + df[f"RA_{c}_s3"]) / 3
    
df["mRA"] = df[[f"RA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [93]:
df[[MODEL_COLUMN]+[f"RA_{c}" for c in CORRUPTIONS] + ["mRA"]]

,model,RA_beamsreducing,RA_brightness,RA_dark,RA_fog,RA_missingcamera,RA_motionblur,RA_pointsreducing,RA_snow,RA_spatialmisalignment,RA_temporalmisalignment,mRA
0,CMT,0.786283,0.984591,0.948377,0.806356,0.973845,0.841198,0.982853,0.833425,0.808825,0.788294,0.875405
1,DeepInteraction,0.654798,0.984079,0.929367,0.583008,0.841994,0.831765,0.974912,0.759058,0.730641,0.768080,0.805770
2,TransFusion,0.632835,0.997788,0.988095,0.754329,0.985460,0.825758,0.968991,0.747553,0.685300,0.776821,0.836293
3,Sparsefusion,0.688631,0.991752,0.963272,0.767100,0.953930,0.847801,0.975849,0.770107,0.714103,0.776760,0.844931
4,BEVfusion,0.676278,0.986749,0.968692,0.752146,0.973871,0.865668,0.973078,0.773936,0.705114,0.741741,0.841727


### To Latex

In [94]:
rn = {f"RA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({MODEL_COLUMN: "Model"})
df_latex = df[[MODEL_COLUMN]+[f"RA_{c}" for c in CORRUPTIONS] + ["mRA"]].rename(columns=rn)
df_latex = df_latex.style.format(precision=3).hide_index().highlight_max(axis=0, props="textbf:--rwrap;")
print(df_latex.to_latex(hrules=True))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{Model} & {Beams Red.} & {Brightness} & {Darkness} & {Fog} & {Missing Cam.} & {Motion Blur} & {Points Red.} & {Snow} & {Spatial Mis.} & {Temporal Mis.} & {mRA} \\
\midrule
CMT & \textbf{0.786} & 0.985 & 0.948 & \textbf{0.806} & 0.974 & 0.841 & \textbf{0.983} & \textbf{0.833} & \textbf{0.809} & \textbf{0.788} & \textbf{0.875} \\
DeepInteraction & 0.655 & 0.984 & 0.929 & 0.583 & 0.842 & 0.832 & 0.975 & 0.759 & 0.731 & 0.768 & 0.806 \\
\textbf{TransFusion} & 0.633 & \textbf{0.998} & \textbf{0.988} & 0.754 & \textbf{0.985} & 0.826 & 0.969 & 0.748 & 0.685 & 0.777 & 0.836 \\
Sparsefusion & 0.689 & 0.992 & 0.963 & 0.767 & 0.954 & 0.848 & 0.976 & 0.770 & 0.714 & 0.777 & 0.845 \\
BEVfusion & 0.676 & 0.987 & 0.969 & 0.752 & 0.974 & \textbf{0.866} & 0.973 & 0.774 & 0.705 & 0.742 & 0.842 \\
\bottomrule
\end{tabular}



### To Markdown

In [95]:
rn = {f"RA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({"model": "Model"})
rn.update({"clean": "Clean"})
df_ra = df.sort_values("mRA", ascending=False)
df_ra = df_ra[[MODEL_COLUMN]+["clean"]+[f"RA_{c}" for c in CORRUPTIONS] + ["mRA"]].rename(columns=rn)

def to_bold_float(cell):
    try:
        return f'**{float(cell):.3f}**'
    except (ValueError, TypeError):
        return cell
    
def to_float(cell):
    try:
        return f'{float(cell):.3f}'
    except (ValueError, TypeError):
        return cell

def md_highlight_max(row):
    try:
        is_max = row == row.max()
        return [to_bold_float(cell) if max_cell else to_float(cell) for cell, max_cell in zip(row, is_max)]
    except (ValueError, TypeError):
        return row

df_markdown = df_ra.apply(md_highlight_max, axis=0).replace(MODELS_TO_LINKS_MD)
print(df_markdown.to_markdown(index=False))

| Model                                                           | Clean     | Beams Red.   | Brightness   | Darkness   | Fog       | Missing Cam.   | Motion Blur   | Points Red.   | Snow      | Spatial Mis.   | Temporal Mis.   | mRA       |
|:----------------------------------------------------------------|:----------|:-------------|:-------------|:-----------|:----------|:---------------|:--------------|:--------------|:----------|:---------------|:----------------|:----------|
| [CMT](https://github.com/junjie18/CMT)                          | 0.729     | **0.786**    | 0.985        | 0.948      | **0.806** | 0.974          | 0.841         | **0.983**     | **0.833** | **0.809**      | **0.788**       | **0.875** |
| [Sparsefusion](https://github.com/yichen928/SparseFusion)       | **0.732** | 0.689        | 0.992        | 0.963      | 0.767     | 0.954          | 0.848         | 0.976         | 0.770     | 0.714          | 0.777           | 0.845     |
| [BEVfusion](https://github

### To HTML

In [96]:
def html_highlight_max(s): 
    if s.dtype == object: 
        is_max = [False for _ in range(s.shape[0])] 
    else: 
        is_max = s == s.max() 
    return ['background: grey; font-weight: bold' if cell else '' for cell in is_max] 


df_ra.style.hide_index().format(precision=3).apply(html_highlight_max)
# print(df_markdown.style.hide_index().format(precision=3).apply(html_highlight_max).to_html())

Model,Clean,Beams Red.,Brightness,Darkness,Fog,Missing Cam.,Motion Blur,Points Red.,Snow,Spatial Mis.,Temporal Mis.,mRA
CMT,0.729,0.786,0.985,0.948,0.806,0.974,0.841,0.983,0.833,0.809,0.788,0.875
Sparsefusion,0.732,0.689,0.992,0.963,0.767,0.954,0.848,0.976,0.770,0.714,0.777,0.845
BEVfusion,0.714,0.676,0.987,0.969,0.752,0.974,0.866,0.973,0.774,0.705,0.742,0.842
TransFusion,0.708,0.633,0.998,0.988,0.754,0.985,0.826,0.969,0.748,0.685,0.777,0.836
DeepInteraction,0.691,0.655,0.984,0.929,0.583,0.842,0.832,0.975,0.759,0.731,0.768,0.806


# Relative Resistance Ability (RRA)

In [97]:
for c in CORRUPTIONS:
    non_baseline = df.loc[df[MODEL_COLUMN] != BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1)
    baseline = df.loc[df[MODEL_COLUMN] == BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1).iloc[0]
    df[f"RRA_{c}"] = (((non_baseline / baseline) - 1)) * 100
    df.loc[df[MODEL_COLUMN] == BASELINE, f"RRA_{c}"] = 0
    
df["mRRA"] = df[[f"RRA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [98]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]]

,model,clean,RRA_beamsreducing,RRA_brightness,RRA_dark,RRA_fog,RRA_missingcamera,RRA_motionblur,RRA_pointsreducing,RRA_snow,RRA_spatialmisalignment,RRA_temporalmisalignment,mRRA
0,CMT,0.7290,18.642197,1.820503,-0.096334,9.398263,2.041012,-0.840834,3.068808,9.887261,17.052673,8.448135,6.942168
1,DeepInteraction,0.6909,-6.361253,-3.551163,-7.215452,-25.037221,-16.385588,-7.077023,-3.107169,-5.148610,0.211752,0.144681,-7.352705
2,TransFusion,0.7084,-7.209880,0.269529,1.146380,-0.552109,0.340169,-5.411524,-1.256293,-4.220172,-3.626257,3.849783,-1.667038
3,Sparsefusion,0.7315,4.263833,2.912805,1.820722,4.429280,0.297049,0.280278,2.685207,1.887020,3.699047,7.227779,2.950302
4,BEVfusion,0.7144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### To LateX

In [99]:
rn = {f"RRA_{c}": NICE_NAMES[c] for c in CORRUPTIONS}
rn.update({MODEL_COLUMN:"Model"})
rn.update({"clean": "Clean"})
df_latex = df[[MODEL_COLUMN]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]].rename(columns=rn)
df_latex = df_latex.style.format(precision=3).hide_index().highlight_max(axis=0, props="textbf:--rwrap;")
print(df_latex.to_latex(hrules=True))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{Model} & {Beams Red.} & {Brightness} & {Darkness} & {Fog} & {Missing Cam.} & {Motion Blur} & {Points Red.} & {Snow} & {Spatial Mis.} & {Temporal Mis.} & {mRRA} \\
\midrule
CMT & \textbf{18.642} & 1.821 & -0.096 & \textbf{9.398} & \textbf{2.041} & -0.841 & \textbf{3.069} & \textbf{9.887} & \textbf{17.053} & \textbf{8.448} & \textbf{6.942} \\
DeepInteraction & -6.361 & -3.551 & -7.215 & -25.037 & -16.386 & -7.077 & -3.107 & -5.149 & 0.212 & 0.145 & -7.353 \\
\textbf{TransFusion} & -7.210 & 0.270 & 1.146 & -0.552 & 0.340 & -5.412 & -1.256 & -4.220 & -3.626 & 3.850 & -1.667 \\
Sparsefusion & 4.264 & \textbf{2.913} & \textbf{1.821} & 4.429 & 0.297 & \textbf{0.280} & 2.685 & 1.887 & 3.699 & 7.228 & 2.950 \\
BEVfusion & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
\bottomrule
\end{tabular}



### To Markdown

In [100]:
df_markdown = df.sort_values("mRRA", ascending=False)
index_to_top = df_markdown.index[df_markdown[MODEL_COLUMN] == BASELINE].to_list()[0]
idx = [index_to_top] + [i for i in df_markdown.index if i!=index_to_top]
df_markdown = df_markdown.loc[idx]
df_markdown = df_markdown[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]].rename(columns=rn)

df_markdown = df_markdown.apply(md_highlight_max, axis=0).replace(MODELS_TO_LINKS_MD)
print(df_markdown.to_markdown(index=False))

| Model                                                           | Clean     | Beams Red.   | Brightness   | Darkness   | Fog       | Missing Cam.   | Motion Blur   | Points Red.   | Snow      | Spatial Mis.   | Temporal Mis.   | mRRA      |
|:----------------------------------------------------------------|:----------|:-------------|:-------------|:-----------|:----------|:---------------|:--------------|:--------------|:----------|:---------------|:----------------|:----------|
| [BEVfusion](https://github.com/mit-han-lab/bevfusion)           | 0.714     | 0.000        | 0.000        | 0.000      | 0.000     | 0.000          | 0.000         | 0.000         | 0.000     | 0.000          | 0.000           | 0.000     |
| [CMT](https://github.com/junjie18/CMT)                          | 0.729     | **18.642**   | 1.821        | -0.096     | **9.398** | **2.041**      | -0.841        | **3.069**     | **9.887** | **17.053**     | **8.448**       | **6.942** |
| [Sparsefusion](https://git